In [16]:
from dotenv import load_dotenv
import os
import json
import time
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import MessagesState, END, StateGraph, START
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.colors import HexColor
from reportlab.pdfbase import pdfmetrics


load_dotenv()


llm = ChatOpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile"
)

# Default brand identity if LLM doesn't provide
DEFAULT_BRAND_IDENTITY = {
    "colors": {"primary": "#0000FF", "secondary": "#000000"},
    "fonts": {"heading": "Helvetica-Bold", "body": "Helvetica"}
}


In [17]:

# Verify environment
print(f"[System] Python executable: {os.environ.get('VIRTUAL_ENV', 'No venv')}\\Scripts\\python.exe")
print(f"[System] Working directory: {os.getcwd()}")
icon_path = os.path.join(os.getcwd(), "icon.png")
print(f"[System] Icon exists: {os.path.exists(icon_path)}")

# ==========================
# Cell 2: Helper Functions
# ==========================
def safe_font(font_name: str, fallback="Helvetica"):
    """Return a font name if registered, else fallback."""
    try:
        pdfmetrics.getFont(font_name)
        return font_name
    except:
        return fallback


[System] Python executable: C:\Users\areeb\Desktop\New folder (2)\MultiAgents_Task\venv\Scripts\python.exe
[System] Working directory: C:\Users\areeb\Desktop\New folder (2)\MultiAgents_Task
[System] Icon exists: True


In [18]:

# ==========================
# Cell 3: Node Definitions
# ==========================
def content_node(state: MessagesState) -> dict:
    query = state["messages"][-1].content
    print(f"[Content Agent] Generating content for query: {query}")
    try:
        prompt = f"""
        For query "{query}":
        1. Generate a title relevant to the query.
        2. Write a ~200-word report on the topic.
        3. Summarize it in ~25 words.
        4. Suggest a brand identity: colors (primary, secondary) and fonts (heading, body).
        Return JSON with 'title', 'report', 'summary', 'brand_identity' fields. No other text.
        """
        response = llm.invoke(prompt, response_format={"type": "json_object"})
        content = json.loads(response.content)
        # Ensure fallback brand identity exists
        content["brand_identity"] = content.get("brand_identity", DEFAULT_BRAND_IDENTITY)
        content["icon_path"] = icon_path if os.path.exists(icon_path) else None

        print(f"[Content Agent] Generated content successfully")
        return {
            "messages": state["messages"] + [HumanMessage(content=json.dumps(content), name="content_generator")],
            "goto": "pdf_generator"
        }
    except Exception as e:
        print(f"[Content Agent] Error: {repr(e)}")
        fallback_content = {
            "title": f"Report on {query}",
            "report": f"No data available for '{query}'. Fallback content generated.",
            "summary": f"No data for '{query}'.",
            "brand_identity": DEFAULT_BRAND_IDENTITY,
            "icon_path": None
        }
        return {
            "messages": state["messages"] + [HumanMessage(content=json.dumps(fallback_content), name="content_generator")],
            "goto": "pdf_generator"
        }


In [19]:

def pdf_node(state: MessagesState) -> dict:
    print("[PDF Agent] Generating PDF...")
    try:
        content = None
        for message in reversed(state["messages"]):
            if message.name == "content_generator":
                content = json.loads(message.content)
                break
        if not content:
            raise ValueError("No content found from Content Agent.")

        # Extract brand identity
        colors = content.get("brand_identity", DEFAULT_BRAND_IDENTITY).get("colors", DEFAULT_BRAND_IDENTITY["colors"])
        fonts = content.get("brand_identity", DEFAULT_BRAND_IDENTITY).get("fonts", DEFAULT_BRAND_IDENTITY["fonts"])
        heading_font = safe_font(fonts.get("heading", "Helvetica-Bold"), fallback="Helvetica-Bold")
        body_font = safe_font(fonts.get("body", "Helvetica"), fallback="Helvetica")

        pdf_path = f"report_{int(time.time())}.pdf"
        doc = SimpleDocTemplate(pdf_path, pagesize=letter)
        styles = getSampleStyleSheet()
        styles.add(ParagraphStyle(name='Heading', fontName=heading_font, fontSize=16, textColor=HexColor(colors["primary"])))
        styles.add(ParagraphStyle(name='Body', fontName=body_font, fontSize=12, textColor=HexColor(colors["secondary"])))

        elements = []
        elements.append(Paragraph(content['title'], styles['Heading']))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph(content['report'], styles['Body']))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph('Summary: ' + content['summary'], styles['Body']))
        elements.append(Spacer(1, 12))

        if content['icon_path'] and os.path.exists(content['icon_path']):
            elements.append(Image(content['icon_path'], width=50, height=50))
        else:
            elements.append(Paragraph('Icon not available', styles['Body']))

        doc.build(elements)
        print(f"[PDF Agent] PDF generated at: {pdf_path}")
        return {
            "messages": state["messages"] + [HumanMessage(content=f"PDF generated: {pdf_path}", name="pdf_generator")],
            "goto": END
        }

    except Exception as e:
        print(f"[PDF Agent] Error: {repr(e)}")
        return {
            "messages": state["messages"] + [HumanMessage(content=f"PDF node error: {repr(e)}")],
            "goto": END
        }


In [20]:

# ==========================
# Cell 4: Workflow
# ==========================
workflow = StateGraph(MessagesState)
workflow.add_node("content_generator", content_node)
workflow.add_node("pdf_generator", pdf_node)
workflow.add_edge(START, "content_generator")
workflow.add_conditional_edges(
    "content_generator",
    lambda state: state.get("goto", "pdf_generator"),
    {"pdf_generator": "pdf_generator", END: END}
)
workflow.add_conditional_edges(
    "pdf_generator",
    lambda state: state.get("goto", END),
    {END: END}
)

graph = workflow.compile()


In [21]:

# Cell: Execute Workflow
# ==========================
query = "Democracy"  # Example query
print(f"[System] Executing workflow for query: {query}")

try:
    events = graph.stream(
        {"messages": [HumanMessage(content=query)]},
        {"recursion_limit": 5}
    )
    for event in events:
        print(event)
        print("----")
except Exception as e:
    print(f"[System] Workflow error: {repr(e)}")

# ==========================

[System] Executing workflow for query: Democracy
[Content Agent] Generating content for query: Democracy
[Content Agent] Generated content successfully
{'content_generator': {'messages': [HumanMessage(content='Democracy', additional_kwargs={}, response_metadata={}, id='37c9710d-5500-4378-9e56-3b261eab7a26'), HumanMessage(content='{"title": "Understanding Democracy", "report": "Democracy is a system of government where power is vested in the people, either directly or through elected representatives. It emphasizes the protection of individual rights and freedoms, ensuring that all citizens have an equal say in the decision-making process. Democratic systems are designed to promote accountability, transparency, and the rule of law. The core principles of democracy include free and fair elections, freedom of speech, and the separation of powers. Effective democracies also rely on an independent judiciary, a free press, and a robust civil society. By providing a framework for resolving con